## http://yann.lecun.com/exdb/mnist/ 에서 손글씨 데이터 추출하기

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]='0' 


In [ ]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(train_images.shape,train_labels.shape)
print(test_images.shape,test_labels.shape)

## 데이터를 학습에 맞도록 크기를 변환
- 1. 각 픽셀의 크기를 $0\le f \le 255$ 에서 $0\le f \le 1$로 크기를 조정해 준다.
- 2. label을 one-hot encoding 을 시켜준다. (해당값을 1로, 그 나머지를 0으로 변환)

In [ ]:
from keras.utils import to_categorical
 
train_images=train_images.astype('float32')/255
test_images=test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## Q3 : CNN의 초기 버전인 LeNet 을 구현하여 학습시켜라. 
- 이 문제를 풀 경우 extra credit 을 부여
- 학생들은 Tensorflow, Keras, Pytouch 등 본인이 원하는 framework을 사용할 수 있다. 

 ![nn](picture.png)

In [ ]:
from keras import models
from keras import layers

train_images=train_images.reshape((60000,28,28,1)) # 마지막 열은 Channel의 갯수, 흑백이므로 channel = 1
test_images=test_images.reshape((10000,28,28,1)) # 마지막 열은 Channel의 갯수, 흑백이므로 channel = 1

model=models.Sequential()

model.add(layers.Conv2D(filters = 6, kernel_size=(5, 5),  padding = "same", activation='relu', input_shape=(28, 28, 1)))
model.add(layers.AveragePooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(16, kernel_size=(5, 5), activation='relu'))
model.add(layers.AveragePooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(120, kernel_size=(5, 5), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history= model.fit(train_images,train_labels,epochs=20, batch_size=100, verbose=2,validation_data=(test_images, test_labels))

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('test accuracy=',test_acc)

In [ ]:
import matplotlib.pyplot as plt


history_dict= history.history

loss = history_dict['loss']
val_loss = history_dict['val_loss']

accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

epochs = range(1, len(loss)+1)

plt.figure(figsize=(10, 4))

plt.subplot(121)


plt.plot(epochs, loss, 'bo',label='training loss')
plt.plot(epochs, val_loss, 'b', label='test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss') 
plt.legend()

plt.subplot(122)


plt.plot(epochs, accuracy, 'bo',label='training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim((0.9,1.02)) 
plt.legend()

plt.tight_layout()
plt.show()

## 생성된 네트워크 모델을 저장하기 

In [ ]:
from keras.models import load_model

model.save('mnist_CNN.h5')

In [ ]:
from keras.models import load_model

model_CNN = load_model('mnist_CNN.h5')

In [ ]:
pred_out = model_CNN.predict(test_images)

In [ ]:
pred_out.shape

In [ ]:
from numpy import argmax

for i in range(5):
    print('True : ' + str(argmax(test_labels[i])) + ', Predict : ' + str(argmax(pred_out[i])))